# FX Swaption Valuation using QuantLib and Monte Carlo Simulation

## Project Overview

This project focuses on the valuation of an FX swaption using QuantLib and Monte Carlo simulation. An FX swaption is a financial derivative that provides the holder the right but not the obligation to enter into a foreign exchange swap contract at a future date, at a predetermined strike rate.

### Objectives

1. **Parameter Setup**: Define the parameters for the FX swaption and Monte Carlo simulation.
2. **Yield Curve Construction**: Build the yield curves for USD and EUR.
3. **FX Model Initialization**: Initialize the FX model using the Black-Scholes-Merton process.
4. **Monte Carlo Simulation**: Run the Monte Carlo simulation to generate paths for the FX rate.
5. **Swaption Valuation**: Calculate the present value of the FX swaption.
6. **Visualization**: Generate plots to visualize the simulation results.

### Tools and Libraries

- **QuantLib**: An open-source library for quantitative finance.
- **Python**: The programming language used for the project.
- **Matplotlib**: A plotting library for Python.

Let's get started!

In [ ]:
!pip install -q QuantLib-Python

## Importing Libraries

In this section, we import the necessary libraries for the project. We use QuantLib for financial calculations, NumPy for numerical operations, and Matplotlib for plotting.

In [ ]:
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt

## Parameter Setup

Here, we define the parameters for the FX swaption and the Monte Carlo simulation. These parameters include the notional amount, strike rate, maturity and tenor of the swaption, volatility, and risk-free rates for both USD and EUR.

In [ ]:
# Parameters for FX Swaption and Monte Carlo simulation
notional = 1000000  # Notional amount
strike = 1.20  # Strike rate (e.g., USD/EUR)
maturity_years = 1  # Maturity of the swaption in years
tenor_years = 5  # Tenor of the underlying swap in years
volatility = 0.15  # Volatility
risk_free_rate_usd = 0.02  # Risk-free rate for USD
risk_free_rate_eur = 0.01  # Risk-free rate for EUR

# QuantLib setup for date and calendar
calculation_date = ql.Date(16, 10, 2023)
ql.Settings.instance().evaluationDate = calculation_date
calendar = ql.NullCalendar()

# Day count and business day conventions
day_count = ql.Actual365Fixed()
business_convention = ql.ModifiedFollowing

## Yield Curve Construction and FX Model Initialization

In this section, we build the yield curves for both the USD and EUR currencies. We also initialize the FX model using the Black-Scholes-Merton process. The yield curves are built using flat forward rates, and the FX model takes into account the spot FX rate, volatility, and the yield curves.


In [ ]:
# Build yield curves for USD and EUR
usd_curve = ql.FlatForward(calculation_date, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate_usd)), day_count)
eur_curve = ql.FlatForward(calculation_date, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate_eur)), day_count)

# Initialize FX model
spot_fx_rate = ql.QuoteHandle(ql.SimpleQuote(strike))
volatility_quote = ql.QuoteHandle(ql.SimpleQuote(volatility))
volatility_curve = ql.BlackConstantVol(calculation_date, calendar, volatility_quote, day_count)
fx_model = ql.BlackScholesMertonProcess(spot_fx_rate, ql.YieldTermStructureHandle(eur_curve), ql.YieldTermStructureHandle(usd_curve), ql.BlackVolTermStructureHandle(volatility_curve))

## Monte Carlo Simulation and Swaption Valuation

Here, we run the Monte Carlo simulation to generate paths for the FX rate. We then use these paths to calculate the present value of the FX swaption. The present value is calculated by taking the average of the payoffs from each path and discounting it back to today's value.


In [ ]:
# Monte Carlo simulation parameters
num_paths = 10000  # Number of simulation paths
timestep = 1  # Time step in years

# Monte Carlo engine for FX swaption
rng = ql.GaussianRandomSequenceGenerator(ql.UniformRandomSequenceGenerator(timestep, ql.UniformRandomGenerator()))
seq = ql.GaussianPathGenerator(fx_model, maturity_years, timestep, rng, False)

# Valuation of the FX swaption
payoffs = []
for i in range(num_paths):
    sample_path = seq.next()
    path = sample_path.value()
    final_value = path[-1]
    payoff = max(final_value - strike, 0)
    payoffs.append(payoff)

# Calculate present value
discount_factor = np.exp(-risk_free_rate_usd * maturity_years)
present_value = np.mean(payoffs) * discount_factor * notional
present_value

76232.1971210339

## Visualization

Finally, we visualize the results of the Monte Carlo simulation. We plot sample paths for the FX rate and create a histogram to show the distribution of the final FX rates. These visualizations help in understanding the behavior of the FX rate and the outcomes of the simulation.


In [ ]:
# Generate some sample paths for visualization
n_sample_paths = 1000
sample_final_values = []
plt.figure(figsize=(10, 6))

for i in range(n_sample_paths):
    sample_path = seq.next()
    path = sample_path.value()
    time = [path.time(j) for j in range(len(path))]
    value = [path[j] for j in range(len(path))]
    sample_final_values.append(value[-1])
    plt.plot(time, value)

# Plot settings
plt.title('Sample Paths for FX Rate (USD/EUR)')
plt.xlabel('Time to Maturity (in years)')
plt.ylabel('FX Rate (USD/EUR)')
plt.show()

# Plot histogram of final FX rates
plt.figure(figsize=(10, 6))
plt.hist(sample_final_values, bins=20, alpha=0.7, color='blue')
plt.title('Histogram of Final FX Rates (USD/EUR)')
plt.xlabel('FX Rate (USD/EUR)')
plt.ylabel('Frequency')
plt.show()